<a href="https://colab.research.google.com/github/pagiorgi/pellusitas/blob/main/creacion_del_pelusa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Retry with simpler cotton texture generation (avoid weird array shapes)
from PIL import Image, ImageFilter, ImageOps, ImageDraw
import numpy as np
import os, zipfile, io, math, random

# Paths
src_path = "pelusa.jpeg"
out_dir = "CottonMonster_UnityAssets"
if os.path.exists(out_dir):
    # clear existing to avoid conflicts
    import shutil
    shutil.rmtree(out_dir)
os.makedirs(out_dir, exist_ok=True)
sprites_dir = os.path.join(out_dir, "Assets", "CottonMonster", "Sprites")
os.makedirs(sprites_dir, exist_ok=True)

# Load source image
img = Image.open(src_path).convert("RGBA")
w, h = img.size

# Create stroke mask
gray = ImageOps.grayscale(img)
gray_np = np.array(gray).astype(np.uint8)
th = 120
mask_np = (gray_np < th).astype(np.uint8) * 255
mask_img = Image.fromarray(mask_np).filter(ImageFilter.MaxFilter(5))
mask_np = np.array(mask_img)

# Flood fill background to get interior
from collections import deque
bg = mask_np.copy()
visited = np.zeros_like(bg, dtype=np.uint8)
h_, w_ = bg.shape
q = deque()
for cx, cy in [(0,0),(w_-1,0),(0,h_-1),(w_-1,h_-1)]:
    q.append((cy,cx))
while q:
    y,x = q.popleft()
    if x<0 or x>=w_ or y<0 or y>=h_: continue
    if visited[y,x]: continue
    if bg[y,x]==255:
        continue
    visited[y,x]=1
    for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
        q.append((y+dy,x+dx))
interior = ((visited==0) & (bg==0)).astype(np.uint8)*255
interior_img = Image.fromarray(interior).convert("L")
interior_img = interior_img.filter(ImageFilter.MedianFilter(5)).filter(ImageFilter.GaussianBlur(radius=2)).point(lambda p: 255 if p>128 else 0)
interior_img = interior_img.resize((w,h))

# Save outline
outline_img = Image.fromarray(mask_np).convert("L")
outline_img.save(os.path.join(sprites_dir, "outline_mask.png"))

# Simpler cotton texture: layered blurred colored circles/noise
def simple_cotton(width, height, base_color=(255,182,193)):
    canvas = Image.new("RGB", (width, height), (255,255,255))
    draw = ImageDraw.Draw(canvas, 'RGBA')
    # paint many soft circles
    for i in range(800):
        radius = random.randint(20, 120)
        x = random.randint(-radius, width+radius)
        y = random.randint(-radius, height+radius)
        color_shift = (random.randint(-20,20), random.randint(-20,20), random.randint(-20,20))
        color = tuple(max(0,min(255, base_color[j]+color_shift[j])) for j in range(3))
        alpha = random.randint(10,40)
        bbox = [x-radius, y-radius, x+radius, y+radius]
        draw.ellipse(bbox, fill=color+(alpha,))
    canvas = canvas.filter(ImageFilter.GaussianBlur(radius=8)).filter(ImageFilter.SMOOTH)
    return canvas

texture = simple_cotton(w, h)
texture.save(os.path.join(sprites_dir, "cotton_texture_sample.png"))

# Create body sprite
body = Image.new("RGBA", (w,h), (0,0,0,0))
tex_rgba = texture.convert("RGBA")
mask_alpha = interior_img.convert("L")
body.paste(tex_rgba, (0,0), mask_alpha)
# outline
outline_large = Image.fromarray(mask_np).convert("L").filter(ImageFilter.GaussianBlur(3)).point(lambda p: 128 if p>10 else 0)
outline_layer = Image.new("RGBA", (w,h), (0,0,0,0))
edge = Image.new("RGBA", (w,h), (200,120,160,0))
outline_layer.paste(edge, (0,0), outline_large)
body = Image.alpha_composite(outline_layer, body)
body.save(os.path.join(sprites_dir, "Body.png"))

# Create stylized limbs and mouth as before
def make_limb(name, size=(300,150), angle=0):
    w2,h2 = size
    img_limb = Image.new("RGBA", (w2,h2), (0,0,0,0))
    d = ImageDraw.Draw(img_limb)
    rect_box = [10, h2//4, w2-10, h2//4*3]
    d.rounded_rectangle(rect_box, radius=h2//4, fill=(0,0,0,0), outline=(30,30,30,255), width=20)
    d.line([(rect_box[0]+10, rect_box[1]+10),(rect_box[0]+40, rect_box[1]+20)], fill=(255,255,255,80), width=6)
    img_limb = img_limb.rotate(angle, expand=True)
    img_limb.save(os.path.join(sprites_dir, f"{name}.png"))
    return img_limb

arm = make_limb("Arm", size=(260,120), angle=-20)
arm2 = make_limb("Arm_R", size=(260,120), angle=20)
leg = make_limb("Leg", size=(220,120), angle=10)
leg2 = make_limb("Leg_R", size=(220,120), angle=-10)

def make_mouth(open=True, size=(220,120)):
    w3,h3 = size
    img = Image.new("RGBA", (w3,h3), (0,0,0,0))
    d = ImageDraw.Draw(img)
    cx,cy = w3//2, h3//2 + (10 if open else 0)
    r = 70
    if open:
        d.ellipse([cx-r, cy-r, cx+r, cy+r], fill=(220,100,130,255))
        d.ellipse([cx-r+18, cy-r+30, cx+r-18, cy+r-10], fill=(20,20,20,255))
        for i in range(-2,3):
            tx = cx + i*20 - 8
            d.rectangle([tx, cy-10, tx+12, cy+10], fill=(255,255,255,255))
    else:
        d.arc([cx-80, cy-20, cx+80, cy+80], start=200, end=340, fill=(30,30,30,255), width=12)
    img.save(os.path.join(sprites_dir, f"Mouth_{'Open' if open else 'Closed'}.png"))
    return img

m_open = make_mouth(True)
m_closed = make_mouth(False)

# README
readme = """Cotton Monster - Unity Assets
--------------------------------

This package contains sprite assets for a cotton-candy character (Cotton Monster).
Files are placed in Assets/CottonMonster/Sprites/

Included:
 - Body.png                   -> Textured cotton-candy body (transparent background)
 - Arm.png, Arm_R.png         -> Stylized arms (png with transparency)
 - Leg.png, Leg_R.png         -> Stylized legs
 - Mouth_Open.png, Mouth_Closed.png -> Mouth states for animation
 - cotton_texture_sample.png  -> The base cotton texture used
 - outline_mask.png           -> Mask extracted from original drawing

How to import into Unity:
 1. Unzip the folder into your Unity project's Assets folder or use Assets > Import New Asset...
 2. Select the PNG files and in the Inspector set:
    - Texture Type: Sprite (2D and UI)
    - Sprite Mode: Single
    - Pixels Per Unit: 100 (adjust as needed)
    - Mesh Type: Full Rect
    - Filter Mode: Bilinear or Point as you like
 3. Install Unity 2D Animation package via Window > Package Manager (search "2D Animation").
 4. Create a new GameObject and add Sprite Renderers for each part (Body, Arm, Arm_R, Leg, Leg_R, Mouth).
 5. Use the Sprite Skin & Bone Editor (2D Animation) to create bones for legs, arms, and mouth.
 6. Create an Animator Controller and add animations: Idle, Run, Eat.
 7. For 'Eat' animation swap the sprite between Mouth_Closed and Mouth_Open (or use Sprite Swap).

Notes:
 - This is an automated generation from your sketch and may need artistic tweaks.
 - If you prefer, I can additionally prepare a Unity-ready prefab YAML, but importing the sprites and using Unity's 2D Animation tools yields best results.

Enjoy!"""

with open(os.path.join(out_dir, "README.txt"), "w") as f:
    f.write(readme)

# Zip
zip_path = "/mnt/data/cotton_monster_unity_assets.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for root, dirs, files in os.walk(out_dir):
        for file in files:
            full = os.path.join(root, file)
            arcname = os.path.relpath(full, out_dir)
            z.write(full, arcname)

zip_path



'/mnt/data/cotton_monster_unity_assets.zip'